# Comprehensive Workflow: From Single-Cell Data to ML-Ready Graph Input

This notebook integrates several scripts to perform a full data processing pipeline. The goal is to convert raw single-cell RNA-seq data (`.h5ad`) into a set of files suitable for training a temporal graph network. The process involves:
1.  **Setup & Configuration**: Defining all paths and importing libraries.
2.  **Node Feature Extraction**: Identifying highly variable genes (HVGs) and extracting their expression values for each cell to serve as node features.
3.  **Edge Inference with Palantir**: Calculating a transition probability matrix between cells to infer directed edges and their weights.
4.  **Edge Filtering and Analysis**: Analyzing the distribution of edge weights and filtering them to retain only the most significant connections.
5.  **Final ML Data Preparation**: Assembling the filtered edges, node labels (cell types), and timestamps into a final CSV file and a corresponding feature matrix (`.npy`).

### 1. Setup and Configuration

**Goal**: Import all necessary libraries and define all input/output paths in a centralized location. This makes the notebook easier to configure and maintain.

**Input**: The required h5ad-related data files can be obtained through the **data_preparation.ipynb** file.

**Output**: Python variables pointing to file paths and created output directories.

In [ ]:
# === Library Imports ===
import os
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import pickle
from palantir.utils import run_diffusion_maps, determine_multiscale_space, compute_kernel
from tqdm.notebook import tqdm

# === Input Data Paths ===
BASE_INPUT_DIR = "./data/human_bone/"
H5AD_PATH = os.path.join(BASE_INPUT_DIR, "HumanBone.h5ad")
CELL_MAP_PATH = os.path.join(BASE_INPUT_DIR, "cell_map.csv")
CELLTYPE_MAP_PATH = os.path.join(BASE_INPUT_DIR, "celltype_map.csv")

# === Output Directory Configuration ===
BASE_OUTPUT_DIR = "./result/human_bone/"
INTERIM_DATA_DIR = os.path.join(BASE_OUTPUT_DIR, "interim_data")
ML_DATA_DIR = os.path.join(BASE_OUTPUT_DIR, "ml_data")
METADATA_DIR = os.path.join(BASE_OUTPUT_DIR, "metadata")

# --- Create output directories if they don't exist ---
os.makedirs(INTERIM_DATA_DIR, exist_ok=True)
os.makedirs(ML_DATA_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

# === Output File Paths ===
# Metadata files
HVG_INDEX_PATH = os.path.join(METADATA_DIR, 'HVG_1000_gene_index.csv')

# Interim data files
FULL_EDGES_PATH = os.path.join(INTERIM_DATA_DIR, 'palantir_full_edges.csv')
FILTERED_EDGES_PATH = os.path.join(INTERIM_DATA_DIR, 'palantir_filtered_edges_gt0.1.csv')

# Final ML data files
NODE_FEATURES_PATH = os.path.join(ML_DATA_DIR, 'HumanBone_node_features_1000.pkl')
ML_DATAFRAME_PATH = os.path.join(ML_DATA_DIR, 'ml_HumanBone.csv')
ML_EDGE_FEATURES_PATH = os.path.join(ML_DATA_DIR, 'ml_HumanBone_edge_features.npy')

### 2. Node Feature Extraction

**Goal**: Generate feature vectors for each cell (node) based on gene expression. We select the top 1000 highly variable genes (HVGs) to create a meaningful and compact feature representation.

**Input**: 
- `HumanBone.h5ad`: The main AnnData object containing the gene expression matrix.
- `cell_map.csv`: A mapping from cell names to unique integer IDs.

**Output**:
- `HVG_1000_gene_index.csv`: A table mapping HVG gene IDs to their index in the feature vector.
- `HumanBone_node_features_1000.pkl`: A pickled dictionary where keys are cell IDs and values are their corresponding HVG expression vectors, grouped by timepoint.

In [3]:
# --- Load the main AnnData object ---
adata = ad.read_h5ad(H5AD_PATH)

# --- Load the cell name to integer ID mapping ---
cell_map_df = pd.read_csv(CELL_MAP_PATH)
cell_name_to_id = dict(zip(cell_map_df['Cell'], cell_map_df['ID']))

# --- Select top 1000 highly variable genes (HVGs) ---
# This identifies the genes with the highest variance relative to their expression level.
sc.pp.highly_variable_genes(adata, n_top_genes=1000, flavor='cell_ranger', subset=True)

# --- Save the index of the selected HVGs for reference ---
hvg_genes = list(adata.var_names)
gene_symbols = adata.var.loc[hvg_genes, 'gene_symbols'].values
hvg_df = pd.DataFrame({
    'Gene_ID': hvg_genes,
    'Gene_Symbol': gene_symbols,
    'HVG_Index': range(len(hvg_genes))
})
hvg_df.to_csv(HVG_INDEX_PATH, index=False)
print(f"✅ HVG index table saved to: {HVG_INDEX_PATH}")
display(hvg_df.head())

# --- Get the expression matrix for the HVGs ---
adata_df = pd.DataFrame(
    adata.X.toarray() if not isinstance(adata.X, np.ndarray) else adata.X,
    index=adata.obs_names,
    columns=adata.var_names
)

# --- Construct the final node feature dictionary: {cell_ID: {timepoint: [expression_vector]}} ---
expression_data = {}
for cell_name, row in tqdm(adata.obs.iterrows(), total=adata.n_obs, desc="Processing Node Features"):
    time = row['Timepoint']
    if pd.isna(time) or cell_name not in cell_name_to_id:
        continue

    cell_id = cell_name_to_id[cell_name]
    expr_vector = adata_df.loc[cell_name].values.astype(float).tolist()
    expression_data[int(cell_id)] = {int(time): expr_vector}

# --- Save the feature dictionary to a .pkl file ---
with open(NODE_FEATURES_PATH, 'wb') as f:
    pickle.dump(expression_data, f)

print(f"\n✅ Node feature data has been saved")
print(f"Total cells processed: {len(expression_data)}")

✅ HVG index table saved to: /home/share/huadjyin/home/s_qinhua2/02code/guozhihan/DynPertub/data_code/cell_development/result/metadata/HVG_1000_gene_index.csv


,Gene_ID,Gene_Symbol,HVG_Index
0,ENSG00000231749,ABCA9-AS1,0
1,ENSG00000138075,ABCG5,1
2,ENSG00000143921,ABCG8,2
3,ENSG00000237741,AC002368.1,3
4,ENSG00000254239,AC002428.2,4


Processing Node Features:   0%|          | 0/88861 [00:00<?, ?it/s]


✅ Node feature data has been saved
Total cells processed: 88861


### 3. Edge Inference with Palantir

**Goal**: Infer relationships (edges) and their strengths (weights) between cells. We use the Palantir method, which involves dimensionality reduction with PCA, followed by diffusion maps to model cell-to-cell transitions.

**Input**:
- `adata` object: The AnnData object, now subsetted to include only HVGs.

**Output**:
- `palantir_full_edges.csv`: A CSV file containing all inferred edges, with columns for `source_cell`, `target_cell`, and `weight` (transition probability).

In [4]:
# --- Standard single-cell preprocessing for Palantir ---
# The data is already subset to HVGs from the previous step.
adata_palantir = adata.copy()
sc.pp.normalize_total(adata_palantir, target_sum=1e4) # Normalize to counts per 10,000
sc.pp.log1p(adata_palantir) # Log-transform the data
sc.pp.scale(adata_palantir) # Scale data to unit variance and zero mean
sc.tl.pca(adata_palantir, n_comps=50) # Perform PCA

# --- Generate diffusion maps and multiscale space ---
# This models the probability of transitioning between cells in the PCA space.
pca_df = pd.DataFrame(adata_palantir.obsm["X_pca"], index=adata_palantir.obs_names)
print("Running diffusion maps...")
dm_res = run_diffusion_maps(pca_df, n_components=30)
print("Determining multiscale space...")
ms_data = determine_multiscale_space(dm_res)

# --- Compute the kernel and transition probability matrix ---
kernel = compute_kernel(ms_data, knn=30, alpha=10)
trans_probs = kernel.multiply(1.0 / np.array(kernel.sum(axis=1)).flatten()[:, None])
trans_probs = trans_probs.tocsr()

# --- Construct the edge list DataFrame ---
# Columns: source_cell, target_cell, weight
rows, cols = trans_probs.nonzero()
weights = trans_probs[rows, cols].A1

cell_names = np.array(pca_df.index)
edges_df = pd.DataFrame({
    "source_cell": cell_names[rows],
    "target_cell": cell_names[cols],
    "weight": weights
})

# --- Save the full edge list to a CSV file ---
edges_df.to_csv(FULL_EDGES_PATH, index=False)

print(f"\n✅ Full edge list has been saved")
print(f"Total edges inferred: {len(edges_df)}")
display(edges_df.head())

Running diffusion maps...
Determining multiscale space...

✅ Full edge list has been saved
Total edges inferred: 3364346


,source_cell,target_cell,weight
0,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM3_HiSeq_5-GCGCAGTCATTCCTCG-1,0.054791
1,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM8_HiSeq_2-CTACATTTCCGAATGT-1,0.001066
2,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM8_HiSeq_4-GTCAAGTAGAGAGCTC-1,0.003759
3,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM4_HiSeq_4-TCTGGAACAATAGAGT-1,0.001271
4,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM6_HiSeq_1-GGAGCAATCTGCGACG-1,0.000569


### 4. Edge Filtering and Analysis

**Goal**: Analyze the distribution of edge weights and filter out weak connections to build a more robust graph. We will keep edges with a transition probability greater than 0.1.

**Input**:
- `palantir_full_edges.csv`: The complete list of inferred edges.

**Output**:
- `palantir_filtered_edges_gt0.1.csv`: A CSV file containing only the edges with `weight > 0.1`.

In [6]:
# --- Analyze the distribution of edge weights ---
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
print(f"Total number of edges before filtering: {len(edges_df)}\n")
print("Number of edges above various thresholds:")
for t in thresholds:
    count = (edges_df["weight"] > t).sum()
    print(f"  - weight > {t}: {count} edges")

# --- Filter edges with weight > 0.1 ---
FILTER_THRESHOLD = 0.1
filtered_edges_df = edges_df[edges_df["weight"] > FILTER_THRESHOLD].copy()

# --- Save the filtered edge list ---
filtered_edges_df.to_csv(FILTERED_EDGES_PATH, index=False)

print(f"\n✅ Filtered edge list (weight > {FILTER_THRESHOLD}) has been saved")
print(f"Number of edges after filtering: {len(filtered_edges_df)}")
display(filtered_edges_df.head())

Total number of edges before filtering: 3364346

Number of edges above various thresholds:
  - weight > 0.001: 3029462 edges
  - weight > 0.005: 2049397 edges
  - weight > 0.01: 1467918 edges
  - weight > 0.05: 403504 edges
  - weight > 0.1: 186044 edges
  - weight > 0.5: 13541 edges

✅ Filtered edge list (weight > 0.1) has been saved
Number of edges after filtering: 186044


,source_cell,target_cell,weight
23,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM8_HiSeq_5-AATCGGTGTGTAATGA-1,0.108461
35,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM6_HiSeq_7-TGAGCATCAAGAGTCG-1,0.227190
39,MantonBM6_HiSeq_5-CTGAAACAGACTAAGT-1,MantonBM8_HiSeq_7-GAACCTACAGGCGATA-1,0.212063
41,MantonBM4_HiSeq_6-TAGAGCTTCACAGTAC-1,MantonBM1_HiSeq_3-GATCGATCAGTAAGCG-1,0.126454
43,MantonBM4_HiSeq_6-TAGAGCTTCACAGTAC-1,MantonBM4_HiSeq_2-GAAACTCCATACGCTA-1,0.184523


### 5. Final ML Data Preparation

**Goal**: Assemble the final machine learning-ready datasets. This involves mapping cell names to their integer IDs, assigning labels (based on source cell type) and timestamps to each edge, and ensuring the final edge list and edge feature matrix are perfectly aligned.

**Input**:
- `palantir_filtered_edges_gt0.1.csv`: The filtered list of cell-to-cell connections.
- `HumanBone.h5ad`: Used to retrieve `Celltype` and `Timepoint` metadata for each cell.
- `cell_map.csv`: Mapping from cell names to integer IDs.
- `celltype_map.csv`: Mapping from cell type names to integer labels.

**Output**:
- `ml_HumanBone.csv`: The final edge data, with columns `u`, `i`, `ts`, `label`, and `idx`.
- `ml_HumanBone_edge_features.npy`: A NumPy array where each row corresponds to an edge in the CSV file and contains its weight.

In [8]:
# === Step 1: Load all necessary mapping files and metadata ===

# --- Load the filtered edges we just created ---
df = pd.read_csv(FILTERED_EDGES_PATH)

# --- Load Cell -> ID mapping (already loaded, but re-assigning for clarity) ---
cell_to_id = cell_name_to_id

# --- Load AnnData to get Celltype and Timepoint metadata ---
adata_meta = ad.read_h5ad(H5AD_PATH)
cell_to_celltype = adata_meta.obs["Celltype"].to_dict()
cell_to_timepoint = adata_meta.obs["Timepoint"].to_dict()

# --- Load Celltype -> Label ID mapping ---
celltype_mapping = pd.read_csv(CELLTYPE_MAP_PATH)
celltype_to_id = dict(zip(celltype_mapping["Celltype"], celltype_mapping["ID"]))

# === Step 2: Create a mapping from Cell ID -> Label ID ===
# This is needed to assign a label to each edge based on its source node's type.
id_to_label = {}
for cell, celltype in cell_to_celltype.items():
    if cell in cell_to_id and celltype in celltype_to_id:
        id_to_label[cell_to_id[cell]] = celltype_to_id[celltype]

# === Step 3: Map columns for the final DataFrame ===
# 'u' for source, 'i' for target, 'ts' for timestamp, 'label' for source node's class.
df["u"] = df["source_cell"].map(cell_to_id)
df["i"] = df["target_cell"].map(cell_to_id)
df["label"] = df["u"].map(id_to_label)
df["ts"] = df["source_cell"].map(cell_to_timepoint)

# === Step 4: Clean, sort, and index the final DataFrame ===
df_ml = df[['u', 'i', 'ts', 'label', 'weight']].copy()

# --- Store original indices before dropping NaNs to keep features aligned ---
df_ml.dropna(inplace=True)
valid_indices = df_ml.index

# --- Sort by timestamp and reset index for the final output ---
df_out = df_ml.sort_values("ts").reset_index(drop=True)

# --- Create the final event index ---
df_out["idx"] = range(1, len(df_out) + 1)

# --- Extract and align edge features --- 
# This is a critical step to ensure the .npy file rows match the .csv file rows.
edge_features = df.loc[valid_indices].sort_values("ts")[['weight']].values

# === Step 5: Save final CSV and NPY files ===

# --- Save the final DataFrame ---
df_to_save = df_out[['u', 'i', 'ts', 'label', 'idx']]
df_to_save.to_csv(ML_DATAFRAME_PATH, index=False)
print(f"✅ Final ML DataFrame saved to: {ML_DATAFRAME_PATH}")
display(df_to_save.head())

# --- Save the aligned edge features ---
np.save(ML_EDGE_FEATURES_PATH, edge_features)
print(f"\n✅ Aligned edge features has been saved to: ml_data folder.")

# === Step 6: Final Validation ===
# Ensure the number of rows in the CSV and the NPY array are identical.
df_rows = len(df_to_save)
feat_rows = edge_features.shape[0]

print(f"\n--- Final Validation ---")
print(f"Rows in {os.path.basename(ML_DATAFRAME_PATH)}: {df_rows}")
print(f"Rows in {os.path.basename(ML_EDGE_FEATURES_PATH)}: {feat_rows}")

if df_rows == feat_rows:
    print("\n✅ Row counts match. Data preparation successful!")
else:
    print("\n❌ ERROR: Row counts do not match. Please check the alignment logic!")

✅ Final ML DataFrame saved to: /home/share/huadjyin/home/s_qinhua2/02code/guozhihan/DynPertub/data_code/cell_development/result/ml_data/ml_HumanBone.csv


,u,i,ts,label,idx
0,44953,48932,19,0,1
1,47645,46539,19,5,2
2,47646,43921,19,5,3
3,47648,43198,19,3,4
4,47649,43267,19,5,5



✅ Aligned edge features has been saved to: ml_data folder.

--- Final Validation ---
Rows in ml_HumanBone.csv: 186044
Rows in ml_HumanBone_edge_features.npy: 186044

✅ Row counts match. Data preparation successful!
